In [47]:
# ======================================================
# README - pdnlS_2D Simulation Notebook
# ======================================================
"""
Simulación 2D de la ecuación pdnlS_2D (parametrically driven nonlinear Schrödinger equation)
con integración RK4-FD, guardado de datos, y animaciones del módulo y fase.

REQUISITOS:
------------
pip install numpy scipy matplotlib

DEPENDENCIAS OPCIONALES:
-------------------------
- FFmpeg (solo si se desea guardar los videos como .mp4)
  Descargar desde https://ffmpeg.org/download.html
  y ajustar la ruta del ejecutable si es necesario:
      matplotlib.rcParams['animation.ffmpeg_path'] = r'C:\\ffmpeg\\bin\\ffmpeg.exe'

Si FFmpeg no está disponible, el código guardará animaciones en formato .gif automáticamente.

SALIDA:
-------
Se generan los siguientes archivos dentro de:
    D:/simulation_data/soliton_control_2D/alpha=.../beta=.../...

1. drive.png        → mapa espacial del bombeo
2. fields_data.npz  → grillas (x, y, t) y campos reales/imaginarios
3. module.mp4   → animación del módulo |A|
4. phase.mp4    → animación de la fase arg(A)

PARA LEER LOS DATOS GUARDADOS:
-------------------------------
data = np.load('fields_data.npz')
x = data['x_grid']
y = data['y_grid']
t = data['t_grid']
U = data['U_real'] + 1j * data['U_imag']

PARAMETROS CLAVE:
-----------------
alpha, beta, gamma_0, mu, nu, sigma   → controlan la ecuación
tf, dt, t_rate                        → control temporal
Lx, Ly, dx, dy                        → control espacial
"""

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import scipy.sparse as sparse
import time
import datetime
import os
matplotlib.rcParams['animation.ffmpeg_path'] = r'C:\ffmpeg\bin\ffmpeg.exe'

def RK4_FD(eq, fields, parameters, grids, dt, Nt, operators, t_rate):
    t_grid = grids[0]  # Time grid
    x_grid = grids[1]  # Spatial grid (x)
    y_grid = grids[2]  # Spatial grid (y)

    fields_history = []
    time_grid = []

    for i in range(Nt - 1):
        t = t_grid[i]  # Current time
        old_fields = fields.copy()  # Ensure copy to avoid overwriting

        # RK4 steps with correct time updates
        k_1 = equations_FD(eq, old_fields, t, x_grid, y_grid, parameters, operators)
        k_2 = equations_FD(eq, old_fields + 0.5 * dt * k_1, t + 0.5 * dt, x_grid, y_grid, parameters, operators)
        k_3 = equations_FD(eq, old_fields + 0.5 * dt * k_2, t + 0.5 * dt, x_grid, y_grid, parameters, operators)
        k_4 = equations_FD(eq, old_fields + dt * k_3, t + dt, x_grid, y_grid, parameters, operators)

        # Update fields using weighted sum
        new_fields = old_fields + dt * (k_1 + 2 * k_2 + 2 * k_3 + k_4) / 6
        fields = new_fields  # Update for next iteration

        # Save history every `t_rate` steps
        if i % t_rate == 0:
            fields_history.append(fields.copy())  # Store a copy to avoid referencing issues
            time_grid.append(t)

    return fields, fields_history, time_grid

def equations_FD(eq, field_slices, t_i, x_grid, y_grid, parameters, operators):
    if eq == 'pdnlS_2D':

        U_1 = field_slices[0]
        U_2 = field_slices[1]

        alpha = parameters[0]
        beta = parameters[1]
        gamma = parameters[2]
        gamma_1 = gamma[0]
        gamma_2 = gamma[1]
        mu = parameters[3]
        nu = parameters[4]

        DDx = operators[0]
        DDy = operators[1]

        ddUx_1 = np.transpose(DDx @ np.transpose(U_1))
        ddUy_1 = DDy @ U_1

        ddUx_2 = np.transpose(DDx @ np.transpose(U_2))
        ddUy_2 = DDy @ U_2

        F = alpha * (ddUx_2 + ddUy_2) + (beta * (U_1 ** 2 + U_2 ** 2) + nu + gamma_2) * U_2 + (gamma_1 - mu) * U_1

        G = -alpha * (ddUx_1 + ddUy_1) - (beta * (U_1 ** 2 + U_2 ** 2) + nu - gamma_2) * U_1 - (gamma_1 + mu) * U_2

        fields = np.array([F, G])
    return fields

def sparse_DD_neumann(Nx, dx):
    data = np.ones((3, Nx))
    data[1] = -2 * data[1]
    diags = [-1, 0, 1]
    D2 = sparse.spdiags(data, diags, Nx, Nx) / (dx ** 2)
    D2 = sparse.lil_matrix(D2)
    # Condiciones de borde de Neumann: ajustar la primera y última fila
    D2[0, 0] = -1 / (dx ** 2)
    D2[0, 1] = 1 / (dx ** 2)
    D2[-1, -1] = -1 / (dx ** 2)
    D2[-1, -2] = 1 / (dx ** 2)
    return D2.tocsr()

In [62]:
# DEFINIENDO DIRECTORIOS Y PARAMETROS
project_name = '/soliton_control_2D'
disc = 'D:/'
eq = 'pdnlS_2D'

alpha = 1
beta = 1
gamma_0 = 0.12
nu = -0.02
sigma = 15
mu = 0.1

# DEFINIENDO GRILLA Y TIEMPOS
ti = 0
tf = 2000     #TIEMPO FINAL DE SIMULACIÓN
dt = 0.05
t_rate = 25

Lx = 140
Ly = 140
dx = 0.5
dy = 0.5

[tmin, tmax, dt] = [0, tf, dt]
[xmin, xmax, dx] = [- Lx / 2,  Lx / 2, dx]
[ymin, ymax, dy] = [- Ly / 2, Ly / 2, dy]
t_grid = np.arange(tmin, tmax + dt, dt)
x_grid = np.arange(xmin, xmax, dx)
y_grid = np.arange(ymin, ymax, dy)
T = tmax
Nt = t_grid.shape[0]
Nx = x_grid.shape[0]
Ny = y_grid.shape[0]
X, Y = np.meshgrid(x_grid, y_grid)

#Z = np.exp((- (X ** 2 + Y ** 2) / (2 * sigma ** 2)))    #SPACE DEPENDENT PUMP
Z = np.ones((Nx, Ny)) # HOMOGENEOUS PUMP
alpha_str = f"{alpha:.{4}f}"
beta_str = f"{beta:.{4}f}"
gamma_str = f"{gamma_0:.{4}f}"
nu_str = f"{nu:.{4}f}"
sigma_str = f"{sigma:.{4}f}"
mu_str = f"{mu:.{4}f}"

file = disc + '/simulation_data' + project_name
subfile = "/alpha=" + alpha_str + "/beta=" + beta_str + "/mu=" + mu_str + "/nu=" + nu_str + "/gamma=" + gamma_str + "/sigma=" + sigma_str
if not os.path.exists(file + subfile):
    os.makedirs(file + subfile)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

pcm1 = ax[0].pcolormesh(x_grid, y_grid, gamma_0 * np.real(Z), cmap="seismic", shading='auto')
cbar1 = plt.colorbar(pcm1, ax=ax[0], shrink=0.8)
cbar1.set_label('Re($Z$)', rotation=0, size=16, labelpad=-30, y=1.12)
ax[0].set_xlabel('$y$', size=16)
ax[0].set_ylabel('$x$', size=16)
ax[0].set_aspect('equal')

pcm2 = ax[1].pcolormesh(x_grid, y_grid, gamma_0 * np.imag(Z), cmap="seismic", shading='auto')
cbar2 = plt.colorbar(pcm2, ax=ax[1], shrink=0.8)
cbar2.set_label('Im($Z$)', rotation=0, size=16, labelpad=-30, y=1.12)
ax[1].set_xlabel('$y$', size=16)
ax[1].set_ylabel('$x$', size=16)
ax[1].set_aspect('equal')

plt.tight_layout()
plt.savefig(file + subfile + '/drive.png', dpi=300)
plt.close()

In [ ]:
# Initial Conditions
delta = np.sqrt(- nu + np.sqrt(gamma_0 ** 2 - mu ** 2))
x_0 = 0
y_0 = 0
#U_1_init = U_2_init = 0.05 * np.random.rand(Ny, Nx)
U_1_init = (np.sqrt(2) * delta / np.cosh(delta * (np.sqrt((X - x_0) ** 2 + (Y - y_0) ** 2) / np.sqrt(alpha)))) * np.real(np.exp(-1j * 0.5 * np.arccos(mu / gamma_0)))
U_2_init = (np.sqrt(2) * delta / np.cosh(delta * (np.sqrt((X - x_0) ** 2 + (Y - y_0) ** 2) / np.sqrt(alpha)))) * np.imag(np.exp(-1j * 0.5 * np.arccos(mu / gamma_0)))

# Empaquetamiento de parametros, campos y derivadas para integración
L = xmax - xmin
D2x = sparse_DD_neumann(Nx, dx)
D2y = sparse_DD_neumann(Ny, dy)
operators = np.array([D2x, D2y])
fields_init = [U_1_init, U_2_init]
grids = [t_grid, x_grid, y_grid]
gamma = [np.real(gamma_0 * Z), np.imag(gamma_0 * Z)]

parameters = [alpha, beta, gamma, mu, nu]

# Midiendo tiempo inicial
now = datetime.datetime.now()
print('Hora de Inicio: ' + str(now.hour) + ':' + str(now.minute) + ':' + str(now.second))
time_init = time.time()

final_fields, fields_history, time_grid = RK4_FD(eq, fields_init, parameters, grids, dt, Nt, operators, t_rate)

time_SIM = time.time()
print("Tiempo de Simulación:" + str(time_SIM - time_init) + ' seg')

# GUARDANDO DATOS
U1_light = np.array(fields_history)[:, 0]
U2_light = np.array(fields_history)[:, 1]
U_complex = U1_light + 1j * U2_light
t_light = time_grid

modulo_light_1 = np.abs(U_complex)
arg_light_1 = np.angle(U_complex)

arg_light_1 = np.where(arg_light_1 > np.pi, arg_light_1 - 2 * np.pi, arg_light_1)
vmax_global = np.max(modulo_light_1)

output_data = os.path.join(file + subfile, 'fields_data.npz')
np.savez_compressed(
    output_data,
    x_grid=x_grid,
    y_grid=y_grid,
    t_grid=t_light,
    U_real=np.real(U_complex),
    U_imag=np.imag(U_complex)
)

# === ANIMACIÓN DEL MÓDULO ============
frames = len(modulo_light_1)
skip = 2
frames_idx = range(0, frames, skip)

os.makedirs(file + subfile, exist_ok=True)
output_path_module = os.path.join(file + subfile, 'module.mp4')

fig, ax = plt.subplots(figsize=(5, 4))
im = ax.imshow(modulo_light_1[0], vmin=0, vmax=vmax_global, cmap="viridis", origin="lower", extent=[xmin, xmax, ymin, ymax], interpolation='nearest')
ax.set_xlabel('$y$', size=16)
ax.set_ylabel('$x$', size=16)
ax.grid(True, color='w', linestyle='--', linewidth=0.5, alpha=0.2)
cbar = fig.colorbar(im, ax=ax, shrink=0.9)
cbar.set_label('$|A|$', rotation=0, size=20, labelpad=-27, y=1.13)

def animate_mod(i):
    im.set_data(modulo_light_1[i])
    return [im]

anim_mod = FuncAnimation(fig, animate_mod, frames=frames_idx, interval=30, blit=False)
writervideo = animation.FFMpegWriter(fps=30, bitrate=3000)
anim_mod.save(output_path_module, writer=writervideo, dpi=120)
plt.close()

# === ANIMACIÓN DE LA FASE ============
output_path_phase = os.path.join(file + subfile, 'phase.mp4')

fig, ax = plt.subplots(figsize=(5, 4))
im2 = ax.imshow(arg_light_1[0], vmin=-np.pi, vmax=np.pi, cmap="hsv", origin="lower", extent=[xmin, xmax, ymin, ymax], interpolation='nearest')
ax.set_xlabel('$y$', size=16)
ax.set_ylabel('$x$', size=16)
ax.grid(True, color='w', linestyle='--', linewidth=0.5, alpha=0.2)
cbar2 = fig.colorbar(im2, ax=ax, shrink=0.9)
cbar2.set_label('$\\mathrm{arg}(A)$', rotation=0, size=20, labelpad=-27, y=1.13)

def animate_phase(i):
    im2.set_data(arg_light_1[i])
    return [im2]

anim_phase = FuncAnimation(fig, animate_phase, frames=frames_idx, interval=30, blit=False)
anim_phase.save(output_path_phase, writer=writervideo, dpi=120)
plt.close()

now = datetime.datetime.now()
print('Hora de Término: ' + str(now.hour) + ':' + str(now.minute) + ':' + str(now.second))
time_fin = time.time()
print(str(time_fin - time_init) + ' seg')

Hora de Inicio: 21:27:38
